# Hybrid Notebook Development Patterns Demo

This notebook demonstrates the core patterns for developing notebooks that run successfully in both **VS Code (Local)** and **Google Colab / Containers**.

**Core Constraints:**
1.  **Environment Agnosticism**: Paths must be dynamic.
2.  **Secret Management**: Secrets must be loaded robustly or injected.
3.  **Dependencies**: Must be installed in the active kernel.
4.  **Process Management**: Clean up background processes.

In [ ]:
# PATTERN 1: Dependency Management
# --------------------------------
# PROBLEM: The notebook kernel is isolated from your host's 'pdm' environment.
# SOLUTION: Use %pip install within the notebook.

%pip install python-dotenv openai

In [ ]:
# PATTERN 2: Filesystem Isolation (The "Container Gap")
# -----------------------------------------------------
# PROBLEM: In Colab/Containers, hardcoded paths like '/Users/name/...' do not exist.
# SOLUTION: Use dynamic paths relative to Path.cwd().

from pathlib import Path

# Dynamic root detection
CURRENT_DIR = Path.cwd()
print(f"Current Working Directory: {CURRENT_DIR}")

# Create a dummy file in the current environment
DATA_FILE = CURRENT_DIR / "demo_data.txt"
DATA_FILE.write_text("This file was created dynamically!", encoding="utf-8")

print(f"✅ Wrote file to: {DATA_FILE}")
print(f"   Exists? {DATA_FILE.exists()}")

In [42]:
import os

# Clear any stale keys from the current kernel session so PATTERN 3 will re-load/prompt.
os.environ.pop("OPENAI_API_KEY", None)
os.environ.pop("LLM_PROVIDER_SERVICE_OPENAI_API_KEY", None)

In [ ]:
# PATTERN 3: Secret Management with Fallback
# ------------------------------------------
# PROBLEM: Host .env files are not automatically mounted in containers.
# SOLUTION: Try to find .env, but allow manual injection or fallback.

import os
from getpass import getpass

from dotenv import load_dotenv


def load_secrets_robustly():
    # 1. Search for .env dynamically (walking up directory tree)
    current = Path.cwd()
    env_path = None
    for _ in range(5):
        if (current / ".env").exists():
            env_path = current / ".env"
            break
        if current.parent == current:
            break
        current = current.parent

    if env_path:
        print(f"✅ Found .env at: {env_path}")
        load_dotenv(env_path, override=True)
    else:
        print("⚠️  .env not found (expected in container environments).")

    # 2. If running inside HuleEdu Docker, map the prefixed service key into
    # OPENAI_API_KEY for this notebook kernel session.
    prefixed_key = os.environ.get("LLM_PROVIDER_SERVICE_OPENAI_API_KEY")
    if prefixed_key and not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = prefixed_key

    # 3. Prompt interactively as a last resort (no secrets committed)
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        api_key = getpass("Enter OPENAI_API_KEY (will not be echoed): ").strip()
        if api_key:
            os.environ["OPENAI_API_KEY"] = api_key

    return api_key


final_key = load_secrets_robustly()
masked = f"{final_key[:4]}...{final_key[-4:]}" if final_key else "None"
print(f"Active OpenAI Key: {masked}")

In [ ]:
# PATTERN 4: Execution Verification
# -------------------------------
# ACTION: Use the injected secret and installed dependency to prove connectivity.


from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    print("❌ No API key found. Skipping API call.")
else:
    try:
        client = OpenAI(api_key=api_key)

        print("🤖 Sending request to OpenAI (gpt-5-nano) [Attempt 3: Debugging empty response]...")

        # Using the requested Nano model with correct params
        response = client.chat.completions.create(
            model="gpt-5-nano",
            messages=[
                {
                    "role": "user",
                    "content": "Say 'Nu förstår du hur du ska använda"
                    "Jupyter notebooks in VS Code!' in German.",
                }
            ],
            max_completion_tokens=2000,
        )

        content = response.choices[0].message.content
        print(f"✅ Response Content: '{content}'")

        # Debug: Print full details if content is empty
        if not content:
            print("\n⚠️ Content was empty! Full response debug:")
            print(response.model_dump_json(indent=2))

    except Exception as e:
        print(f"❌ API Call Failed: {e}")
        print("Note: If you see 'max_tokens' error, try reloading the notebook window.")